環境はRNA_sqe

In [ ]:
conda activate RNAseq

seqkitのインストールコマンド（一度インストールすれば次回から不要）

In [ ]:
mamba install -c bioconda seqkit

fastqファイルの中身を見る

In [ ]:
seqkit stats *_001.fastq.gz > statistics.tsv

seqkit statsコマンドでfastqファイルをタブで分ける/ statistics.tvsは出力されるファイル名（任意に変更可能）

In [ ]:
seqkit stats -T *_001.fastq.gz > statistics.tsv

出力されたデータをexcelで開き表を成型する。その後適切なファイル名に変更し保存する。

アダプター配列を検索して、それを抜いた配列を表示させる
"|"　コマンドを左から順番に実行させる
"-v"　検索でヒットした配列を抜く
"@"　


In [ ]:
grep  -v "@" R1.fastq |grep "ATTCCATA" |less

"wc" ヒットした行数を表示する
wc -l

In [ ]:
grep  -v "@" R1.fastq |grep "ATTCCATA" |wc -l

fastpを使うとリードの品質が低い領域のトリミングとアダプター除去を
１度に実行できる（ブログ）。Phred qualityスコア20以下のリード領域をトリミング”-q 20”
-t, -T 1は塩基の左右の一塩基を強制的に取り除いている（NGSだと最初と最後の一文字の精度が低いため）
-w 8 CUPを8個使うという事

In [ ]:
fastp -i WT-2_R1_001.fastq.gz -I WT-2_R2_001.fastq.gz -3 -o out_pair1.fq.gz -O out_pair2.fq.gz \-h report.html -j report.json -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1

出力されたreporファイル(HTMLファイル)を開く

より精度を高めother adapter sequenceを減らすためアダプター配列(pdfに書いてある)を指定して抜くコマンド

In [ ]:
fastp -i WT-2_R1_001.fastq.gz -I WT-2_R2_001.fastq.gz -3 -o out_pair1.fq.gz -O out_pair2.fq.gz \-h report.html -j report.json -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1 --adapter_sequence ATTCCATA --adapter_sequence_r2 CGCTCCAC  

同様にHTMLのレポートを開いてother adapter sequenceが減っていることを確認する
（今回は何故か増えた）

他の物でも同じように実行する

In [ ]:
fastp -i dg237_R1_001.fastq.gz -I dg237_R2_001.fastq.gz -3 -o out_pair1_dg237_R1.fq.gz -O out_pair2_dg237_R2.fq.gz \-h report_dg237.html -j report_dg237.json -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1 --adapter_sequence AGGTCAGA --adapter_sequence_r2 CGCTCCAC

In [ ]:
 fastp -i BR10PR22_R1_001.fastq.gz -I BR10PR22_R2_001.fastq.gz -3 -o out_pair1_BR10PR22_R1.fq.gz -O out_pair2_BR10PR22_R2.fq.gz \-h report_BR10PR22.html -j report_BR10PR22.json -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1 --adapter_sequence GACGAGAT --adapter_sequence_r2 ATTAACAA

いったんここまででゲノムデータの掃除が終わり

ここからDNAのマッピングに入る
まずはインデックスを引く
インデックスをつける作業は1つのリファレンスゲノムに対して1回だけでOK

In [ ]:
bowtie2-build GCA_002142475.1.fna bowite2_index

インデックスファイルに対して、各サンプル（1サンプル当たり2つのファイル、先ほどきれいに整えたデータ、拡張子は.gzのままで大丈夫）をindexを用いてアラインメントさせる

In [ ]:
bowtie2 --sensitive -x bowite2_index -1 out_pair1.fq.gz -2 out_pair2.fq.gz |samtools sort -@ 8 - >sample1_WT-2.bam

出力されたbamファイルもインデックスをつけてあげる

In [ ]:
samtools index -@ 4 sample1_WT-2.bam

In [ ]:
最後にマッピング率を調べる。

In [ ]:
samtools flagstats sample1_WT-2.bam